In [1]:
import pandas as pd
import pyomo.environ as pyo
from random import sample
from cleaning_mock import create_demand

In [2]:
nodes = pd.read_pickle("../../data/mock/mock_nodes.pkl")
channels = pd.read_pickle("../../data/mock/mock_channels.pkl")

In [3]:
model = pyo.ConcreteModel(name="Min cost flow problem")
model.NODES = pyo.Set(initialize=nodes.index)
model.CHANNELS = pyo.Set(initialize=[(channels.loc[i, "node1_pub"], channels.loc[i, "node2_pub"]) for i in channels.index])

In [4]:
nodes = create_demand(nodes, 156033)

Transaction of 156033 sats.
Sender: LRHZSDVIUT.com
Receiver: IRHDZFIROE.com.


In [5]:
model.x = pyo.Var(model.CHANNELS, domain=pyo.Binary)
model.a = pyo.Var(model.CHANNELS, domain=pyo.NonNegativeReals, bounds=(0, max(nodes["demand"])))

In [6]:
channels.reset_index(inplace=True)
channels.set_index(["node1_pub", "node2_pub"], inplace=True)
channels.sort_index(inplace=True)

In [7]:
def objective_function(model: pyo.ConcreteModel):
    return sum(model.x[i] * channels.loc[i, "base_fee"] for i in model.CHANNELS) + sum(model.a[i] * channels.loc[i, "rate_fee"] for i in model.CHANNELS)

model.totalCost = pyo.Objective(rule=objective_function(model), sense=pyo.minimize)

In [8]:
## Single path constrain
def number_path_rule(model: pyo.ConcreteModel, n):
    outgoing = [model.x[(i, j)] for i, j in channels.index if i == n]
    incoming = [model.x[(i, j)] for i, j in channels.index if j == n]
    return sum(incoming) == sum(outgoing)

model.NumberPathConstraint = pyo.Constraint(model.NODES, rule=number_path_rule)

In [9]:
def capacity_constraint(model: pyo.ConcreteModel, a, b):
    return model.a[(a, b)] <=  channels.loc[(a, b), "capacity"] * model.x[(a, b)]

model.CapacityConstraint = pyo.Constraint(model.CHANNELS, rule=capacity_constraint, name="Capacity constraint")

In [10]:
channels.reset_index(inplace=True)
channels.set_index("channel_id", inplace=True)

def flow_balance_constraint(model: pyo.ConcreteModel, n: str):
    InFlow = sum(model.a[(channels.loc[a, "node1_pub"], channels.loc[a, "node2_pub"])] for a in nodes.loc[n, 'incoming_channels'])
    OutFlow = sum(model.a[(channels.loc[a, "node1_pub"], channels.loc[a, "node2_pub"])] for a in nodes.loc[n, 'outgoing_channels'])
    return  OutFlow + nodes.loc[n, "demand"] == InFlow

model.FlowBalanceConstraint = pyo.Constraint(model.NODES, rule=flow_balance_constraint, name="Flow balance constrain")

channels.reset_index(inplace=True)
channels.set_index(["node1_pub", "node2_pub"], inplace=True)
channels.sort_index(inplace=True)        

In [11]:
opt = pyo.SolverFactory('cbc')
results = opt.solve(model, tee=True)

if (results.solver.status == pyo.SolverStatus.ok) and (results.solver.termination_condition == pyo.TerminationCondition.optimal):
    print('\nOptimal solution found')
elif results.solver.termination_condition == pyo.TerminationCondition.feasible:
    print('\nFeasible but not proven optimal solution found')
elif results.solver.termination_condition == pyo.TerminationCondition.infeasible:
    raise Exception("The model is infeasible")
else:
    print('\nSolver Status: ',  results.solver.status)
    raise Exception(results.solver.status)

print('\nObject function value = ', model.Objective())



Welcome to the CBC MILP Solver 
Version: 2.10.8 
Build Date: May  9 2022 

command line - /usr/bin/cbc -printingOptions all -import /tmp/tmprrnsvmv6.pyomo.lp -stat=1 -solve -solu /tmp/tmprrnsvmv6.pyomo.soln (default strategy 1)
Option for printingOptions changed from normal to all
Presolve 879 (-3) rows, 1378 (-2) columns and 4026 (-10) elements
Statistics for presolved model
Original problem has 690 integers (690 of which binary)
Presolved problem has 689 integers (689 of which binary)
==== 77 zero objective 511 different
==== absolute objective values 511 different
==== for integers 1 zero objective 497 different
==== for integers absolute objective values 497 different
===== end objective counts


Problem has 879 rows, 1378 columns (1301 with objective) and 4026 elements
There are 6 singletons with objective 
Column breakdown:
0 of type 0.0->inf, 689 of type 0.0->up, 0 of type lo->inf, 
0 of type lo->up, 0 of type free, 0 of type fixed, 
0 of type -inf->0.0, 0 of type -inf->up, 689 

In [12]:
from decimal import Decimal
DF_channels = pd.DataFrame()
c = 0
for index, value in model.a.extract_values().items():
    DF_channels.loc[c, "source"] = index[0]
    DF_channels.loc[c, "destination"] = index[1]
    DF_channels.loc[c, "capacity"] = channels.loc[index, "capacity"]
    DF_channels.loc[c, "amount"] = Decimal(value)
    c += 1
    
DF_channels_pos = DF_channels[DF_channels["amount"]!=0]
DF_channels_pos

,source,destination,capacity,amount
265,LRHZSDVIUT,RQTUFVOQNG,2438839.0,156033
362,PGFZUBGVTF,TPNSRRMLMO,6165000.0,156033
462,RQTUFVOQNG,PGFZUBGVTF,2798488.0,156033
520,TPNSRRMLMO,IRHDZFIROE,9675139.0,156033


In [13]:
DF_channels[DF_channels["amount"]> DF_channels["capacity"]]

,source,destination,capacity,amount


In [14]:
#nodes.reset_index(inplace=True)
#DF_nodes = nodes[nodes["pub_key"].isin(DF_channels.loc[DF_channels["amount"] > 0, "source"].tolist()) | nodes["pub_key"].isin(DF_channels.loc[DF_channels["amount"] > 0, "destination"].tolist())]
#nodes.set_index("pub_key", inplace=True)
#
#DF_nodes

In [15]:
DF_fixed = pd.DataFrame()
c = 0
for index, value in model.x.extract_values().items():
    DF_fixed.loc[c, "source"] = index[0]
    DF_fixed.loc[c, "destination"] = index[1]
    DF_fixed.loc[c, "used"] = Decimal(value)
    c += 1
    
DF_fixed_pos = DF_fixed[DF_fixed["used"]!=0]

In [16]:
# Compare in order to catch not used channels that are selected by the fixed charge constrain
intersection = DF_fixed_pos.merge(DF_channels_pos, on=["source", "destination"], how="outer")
intersection

,source,destination,used,capacity,amount
0,EPCVSGNLFU,LRHZSDVIUT,1,NaN,NaN
1,GAIHFAVBIB,EPCVSGNLFU,1,NaN,NaN
2,IRHDZFIROE,UZHIMMBPBI,1,NaN,NaN
3,LRHZSDVIUT,RQTUFVOQNG,1,2438839.0,156033
4,PGFZUBGVTF,TPNSRRMLMO,1,6165000.0,156033
5,RQTUFVOQNG,PGFZUBGVTF,1,2798488.0,156033
6,TPNSRRMLMO,IRHDZFIROE,1,9675139.0,156033
7,UZHIMMBPBI,GAIHFAVBIB,1,NaN,NaN
